# PolyLogToolkit

*LogToolkit* is a script implemented to support symbolic computations of multiple polylogarithms, detailed in Haoran Li's dissertation [Hopf Algebra of Multiple Polylogarithms and Its Associated One Forms](https://lihaoranicefire.github.io/math/LogToolKit/HopfAlgebraOfMultiplePolylogarithmsAndItsAssociatedOneForms.pdf)

Elements in $\mathbb I^{\text{Symb}}$

$I(a_{i_0}; 0^{n_0-1}, a_{i_1}, 0^{n_1-1}, \cdots, a_{i_m}, 0^{m_d-1}; a_{i_{m+1}})\leftrightsquigarrow (i_0, n_0, i_1, n_1, \cdots, i_m, n_m, i_{m+1})$

for example, in depth $2$, $I(a_0; 0^{1-1}, a_1, 0^{3-1}, a_2, 0^{2-1}; a_3)\leftrightsquigarrow(0,1,1,3,2,2,2)$

In [ ]:
def IteratedIntegral():
    def __init__(self, args, depth):
        self.args = args
        self.depth = depth

In [ ]:
IteratedIntegral([0, 1, 1, 3, 2, 2, 2], 2)

IteratedItegral([0, 1, 1, 3, 2, 2, 2], 2)